Questions for TA: 

Is shp file path setup correctly?

In [190]:
import geopandas as gp
import pandas as pd
import os
import json
import pylab as pl
import csv
import urllib2
import numpy as np
# import zipfile as zp
import statsmodels.api as sm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Importing Energy_and_Water_Data_Disclosure_for_Local_Law_84 Data

In [191]:
url = 'https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv?accessType=DOWNLOAD'
response = urllib2.urlopen(url)
energy = gp.GeoDataFrame.from_csv(response)

# creating simpler variable name for later
energy['BBL'] = energy['NYC Borough, Block, and Lot (BBL)']
energy['EUI'] = energy['Site EUI(kBtu/ft2)']

print energy.columns

Index([u'NYC Borough, Block, and Lot (BBL)', u'Co-reported BBL Status',
       u'BBLs Co-reported',
       u'Reported NYC Building Identificaiton Numbers (BINs)',
       u'Street Number', u'Street Name', u'Borough', u'Zip Code',
       u'DOF Benchmarking Submission Status', u'Site EUI(kBtu/ft2)',
       u'Weather Normalized Site EUI(kBtu/ft2)', u'Source EUI(kBtu/ft2)',
       u'Weather Normalized Source EUI(kBtu/ft2)',
       u'Municipally Supplied Potable Water - Indoor Intensity (gal/ft²)',
       u'Automatic Water Benchmarking Eligible', u'Reported Water Method',
       u'ENERGY STAR Score', u'Total GHG Emissions(MtCO2e)',
       u'Direct GHG Emissions(MtCO2e)', u'Indirect GHG Emissions(MtCO2e)',
       u'Reported Property Floor Area (Building(s)) (ft²)',
       u'DOF Property Floor Area (Buildngs and Parking)(ft2)',
       u'Primary Property Type - Self Selected', u'DOF Number of Buildings',
       u'BBL', u'EUI'],
      dtype='object')


### Importing PLUTO Data

In [192]:
# url2 = 'http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/mn_mappluto_15v1.zip'
# response2 = urllib2.urlopen(url2)
# #  = zp.ZipFile.open(name[, mode[, pwd]])

# zipped = zp.ZipFile(response2, 'r')
# pluto = zipped.open('Manhattan/MNMapPLUTO.shp')

In [193]:
pluto = gp.GeoDataFrame.from_file(os.getenv('PUI2015')+'/HW5/Manhattan/MNMapPLUTO.shp')
print pluto.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AllZoning1',
       u'AllZoning2', u'AreaSource', u'AssessLand',  u'AssessTot',
              u'BBL',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',   u'BoroCode',    u'Borough',
         u'BsmtCode',  u'BuiltCode',   u'BuiltFAR',     u'CB2010',
               u'CD',     u'CT2010',    u'ComArea',    u'CommFAR',
          u'CondoNo',    u'Council',  u'EDesigNum',  u'Easements',
       u'ExemptLand',  u'ExemptTot',        u'Ext',   u'FacilFAR',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
         u'HistDist', u'IrrLotCode',    u'LandUse',   u'Landmark',
              u'Lot',    u'LotArea',   u'LotDepth',   u'LotFront',
          u'LotType',  u'LtdHeight', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',   u'Overlay1',
         u'Overlay2',  u'OwnerName',  u'OwnerType', u'PLUTOMapID',
       u'PolicePrct',   u'ProxCode',    u'ResArea',   u'ResidF

In [194]:
# from pandas.tools.plotting import scatter_matrix
# scatter_matrix (energy, s=300, figsize=(12, 12), diagonal='kde')

### Data Cleanup

In [195]:
def canconvert(mydata):
    try :
        float(mydata)
        return True
    except ValueError:
        return False
    
fun = np.vectorize(canconvert)

In [196]:
index_units = fun(pluto['UnitsTotal'])
pluto['UnitsTotal'][~index_units] = float('NaN')
pluto['UnitsTotal'] = [float(x) for x in pluto['UnitsTotal']]

index_pbbl = fun(pluto['BBL'])
pluto['BBL'][~index_pbbl] = float('NaN')
pluto['BBL'] = [str(x) for x in pluto['BBL']]

index_area = fun(energy['Reported Property Floor Area (Building(s)) (ft²)'])
energy['Reported Property Floor Area (Building(s)) (ft²)'][~index_area] = float('NaN')
energy['Reported Property Floor Area (Building(s)) (ft²)'] = [float(x) for x in energy['Reported Property Floor Area (Building(s)) (ft²)']]

index_use = fun(energy['EUI'])
energy['EUI'][~index_use] = float('NaN')
energy['EUI'] = [float(x) for x in energy['EUI']]

index_ebbl = fun(energy['BBL'])
energy['BBL'][~index_ebbl] = float('NaN')
energy['BBL'] = [str(x) for x in energy['BBL']]

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning

In [197]:
# print energy.EUI
# print energy.BBL
# print energy['Reported Property Floor Area (Building(s)) (ft²)']
# print pluto.BBL
# print pluto.UnitsTotal

###Merging the two data sets using the BBL

In [239]:
joined = pd.merge(energy,pluto,on=['BBL'],how='inner')
len(joined.BBL)

6545

In [240]:
# adding new column with total consumption/building
joined.total = []
joined.total = joined.EUI*joined['Reported Property Floor Area (Building(s)) (ft²)']

# print joined.total

### Additional cleanup - removing outliers

In [236]:
joined = joined[joined.total>0]
len(joined.BBL)

5083

In [241]:
# plot(joined.UnitsTotal, joined.total)

###Finding linear regression model

In [243]:
X = sm.add_constant(np.log10(joined.total))
# print X
model = sm.OLS(np.log10(joined.total), X, missing='drop').fit() # ignores entires where x or y is NaN
# fit = model.fit()
# return fit.params[1], fit.params[0], fit # could also return stderr in each via fit.bse

In [130]:
# fit and plots here

In [129]:
# try making a fit for energy vs units AND a fit for units vs energy. tell me which is better.
# look at the fit summary: which is better? could you do a likelihood ratio test here? why or why not??

In [ ]:
# now fit a second degree curve to it: ax^2+bx+c.
# np.polyfit(x, y, 2)
# or statmodels.formula.api passing a formula like
# formula = 'y ~ 1 + x + I(x ^ 2.0)'   

In [131]:
# look at the fit summary: which is better? could you do a likelihood ratio test here? why or why not??